In [77]:
import numpy as np
import pandas as pd
import tensorflow as tf

# 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

>- パラメータの更新
- 隠れ層の定義
- 損失関数の選択
- 活性化関数の選択


# 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

## まとめ
tensorでは計算式の関数を使うことで、スクラッチのように計算を関数化しなくても利用できる.
そのほか、NN用の活性化関数を呼び出せる。最終的な実行方法もSessionを利用しないといけない点も大きなちがい。

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

### データセットの用意 
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

In [157]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを分類する
"""
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルをOne-Hot数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1

y = y.astype(np.int)[:, np.newaxis]


#y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(y_train.shape)

(96, 3)


In [158]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [159]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 20
num_epochs = 200
n_hidden1 = 50
n_hidden2 = 50
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

In [160]:
X  = tf.placeholder('float',[None, 4])
Y = tf.placeholder('float', [None, 3])
#Y = tf.placeholder('int', [None])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def iris_net(x):
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes])),
    }
    baias = {
        'b1' : tf.Variable(tf.random_normal([n_hidden1])),
        'b2' : tf.Variable(tf.random_normal([n_hidden2])),
        'b3' : tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), baias['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), baias['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) +  baias['b3']
    return layer_output

# ネットワーク読み込み
logits = iris_net(X)
#目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
#loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logit(labels=Y, logits=logits))
#最適化
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
#推定結果(多値分類用：予測値の各行で最大となるインデックスを一つ返す)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
#指標計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

In [161]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        if  epoch % 10 == 0:
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 553.3831, val_loss : 622.1496, acc : 0.375, val_acc : 0.333
Epoch 10, loss : 126.8872, val_loss : 124.8944, acc : 0.375, val_acc : 0.375
Epoch 20, loss : 1.0683, val_loss : 5.0069, acc : 0.812, val_acc : 0.583
Epoch 30, loss : 0.1220, val_loss : 1.3196, acc : 0.938, val_acc : 0.792
Epoch 40, loss : 0.2317, val_loss : 1.3207, acc : 0.938, val_acc : 0.875
Epoch 50, loss : 0.2390, val_loss : 1.2995, acc : 0.938, val_acc : 0.875
Epoch 60, loss : 0.2375, val_loss : 1.2574, acc : 0.938, val_acc : 0.875
Epoch 70, loss : 0.2214, val_loss : 1.1645, acc : 0.938, val_acc : 0.875
Epoch 80, loss : 0.1936, val_loss : 1.0953, acc : 0.938, val_acc : 0.875
Epoch 90, loss : 0.1504, val_loss : 1.0351, acc : 0.938, val_acc : 0.917
Epoch 100, loss : 0.1079, val_loss : 0.9784, acc : 0.938, val_acc : 0.917
Epoch 110, loss : 0.0727, val_loss : 0.9255, acc : 0.938, val_acc : 0.917
Epoch 120, loss : 0.0471, val_loss : 0.8805, acc : 0.938, val_acc : 0.917
Epoch 130, loss : 0.0310, val_loss : 0.84

# 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。



In [170]:
df = pd.read_csv('train.csv')

### データの準備

In [171]:
X = df.loc[:, ['GrLivArea', 'YearBuilt']]
y = df['SalePrice']
X = np.array(X).astype(np.float64)
y = np.array(y).astype(np.float64)
# 値大きいので対数変換
y = np.log(y)
y = y.reshape(-1, 1)
# 標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X  = sc.fit_transform(X)

In [172]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [179]:
# ハイパーパラメータの設定
learning_rate = 0.0001
batch_size = 20
num_epochs = 200
n_hidden1 = 50
n_hidden2 = 100
n_out = 1
# 特徴量の数
n_input = X_train.shape[1]
n_samples = X_train.shape[0]

In [180]:
X  = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, 1])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def houseprice_net(x):
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_out])),
    }
    baias = {
        'b1' : tf.Variable(tf.random_normal([n_hidden1])),
        'b2' : tf.Variable(tf.random_normal([n_hidden2])),
        'b3' : tf.Variable(tf.random_normal([n_out]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), baias['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), baias['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) +  baias['b3']
    return layer_output

# ネットワーク読み込み
logits = houseprice_net(X)
#目的関数
loss_op = tf.losses.mean_squared_error(labels = Y, predictions = logits)
# 最適化
train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_op)

correct_pred = logits
#指標計算
MSE = tf.reduce_mean(tf.square(Y - logits))

# Initializing
init = tf.initialize_all_variables()

In [181]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_mse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y :mini_batch_y})
            loss, mse = sess.run([loss_op, MSE], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_loss /= n_samples
        #val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        if epoch % 10 ==0:
            val_loss, val_mse = sess.run([loss_op, MSE], feed_dict={X: X_val, Y: y_val})
            print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}, val_mse : {:.3f}".format(epoch, loss, val_loss, mse, val_mse))
            #print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))
    y_pred = sess.run(correct_pred, feed_dict={X: X_test, Y: y_test})
    for i in range(9):
        print("実際の不動産価格: ", y_test[i,0], "  予測した不動産価格: ", y_pred[i,0])


Epoch 0, loss : 21.8362, val_loss : 26.0562, mse : 21.836, val_mse : 26.056
Epoch 10, loss : 2.2396, val_loss : 4.7773, mse : 2.240, val_mse : 4.777
Epoch 20, loss : 0.7010, val_loss : 2.5603, mse : 0.701, val_mse : 2.560
Epoch 30, loss : 0.3732, val_loss : 1.7530, mse : 0.373, val_mse : 1.753
Epoch 40, loss : 0.2385, val_loss : 1.4002, mse : 0.238, val_mse : 1.400
Epoch 50, loss : 0.1751, val_loss : 1.1980, mse : 0.175, val_mse : 1.198
Epoch 60, loss : 0.1374, val_loss : 1.0690, mse : 0.137, val_mse : 1.069
Epoch 70, loss : 0.1158, val_loss : 0.9809, mse : 0.116, val_mse : 0.981
Epoch 80, loss : 0.1036, val_loss : 0.9096, mse : 0.104, val_mse : 0.910
Epoch 90, loss : 0.0929, val_loss : 0.8503, mse : 0.093, val_mse : 0.850
Epoch 100, loss : 0.0835, val_loss : 0.8040, mse : 0.083, val_mse : 0.804
Epoch 110, loss : 0.0775, val_loss : 0.7681, mse : 0.077, val_mse : 0.768
Epoch 120, loss : 0.0743, val_loss : 0.7296, mse : 0.074, val_mse : 0.730
Epoch 130, loss : 0.0725, val_loss : 0.6943, 

# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [147]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [148]:
# データの前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [149]:
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]
print(X_test.shape)

(10000, 1, 28, 28)


In [150]:
# One-Hot-encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_o = enc.fit_transform(y_train[:, np.newaxis])
y_test_o = enc.fit_transform(y_test[:, np.newaxis])

print(y_train.shape)
print(y_train_o.shape)

(60000,)
(60000, 10)


In [151]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_o, test_size=0.2)
print(X_train.shape)
print(X_val.shape)

(48000, 1, 28, 28)
(12000, 1, 28, 28)


In [152]:
# 重み計算
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
# バイアス計算
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#　畳み込み層の定義
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')
# maxpool層の定義
def max_pool_2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME' )

In [153]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 20
num_epochs = 5
n_hidden1 = 32
n_hidden2 = 64
H = X_train.shape[2]
W = X_train.shape[3]
img_size = H*W
#channel = X_train.shape[1]
n_samples = X_train.shape[0]
out_put = 10

In [154]:
X  = tf.placeholder(tf.float32, [None, 1, H, W])
Y = tf.placeholder(tf.float32, [None, out_put])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def mnist_net(x):
    #重みとバイアスの宣言
    weights = {
        'w1': weight_variable([5, 5, 1, n_hidden1]),
        'w2': weight_variable([5, 5, n_hidden1, n_hidden2]),
        'w_fc1':weight_variable([7*7*n_hidden2, 1024]),
        'w_fc2': weight_variable([1024, out_put]),
    }
    baias = {
        'b1' : bias_variable([n_hidden1]),
        'b2' : bias_variable([n_hidden2]),
        'b_fc1': bias_variable([1024]),
        'b_fc2' : bias_variable([out_put])
    }
    
    # xの型変換
    x = tf.reshape(x, [-1,28,28,1])
    #layer1
    layer_1 = conv2d(x, weights['w1'])
    layer_1 = tf.nn.relu(layer_1 + baias['b1'])
    #pooling1
    h_pool1 = max_pool_2d(layer_1)
    #layer2
    layer_2 = conv2d(h_pool1, weights['w2'])
    layer_2 = tf.nn.relu(layer_2 + baias['b2'])
    # pooling2
    h_pool2 = max_pool_2d(layer_2)
    # 全結合用のflat変換
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    # fullconv1
    layer_3 = tf.matmul(h_pool2_flat, weights['w_fc1']) +  baias['b_fc1']
    layer_3 = tf.nn.relu(layer_3)
    # fullconv2
    layer_output = tf.matmul(layer_3, weights['w_fc2']) + baias['b_fc2']
    return layer_output


In [155]:
# ネットワーク読み込み
logits = mnist_net(X)
#目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
#最適化（Adam）
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
#推定結果(多値分類用：予測値の各行で最大となるインデックスを一つ返す)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
#指標計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

# Initializing
init = tf.initialize_all_variables()

In [156]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    # Yにはワンホットエンコーディングしたデータ利用
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_o})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.0106, val_loss : 0.0645, acc : 1.000, val_acc : 0.980
Epoch 1, loss : 0.0039, val_loss : 0.0460, acc : 1.000, val_acc : 0.986
Epoch 2, loss : 0.0030, val_loss : 0.0534, acc : 1.000, val_acc : 0.984
Epoch 3, loss : 0.0016, val_loss : 0.0672, acc : 1.000, val_acc : 0.984
Epoch 4, loss : 0.0000, val_loss : 0.0546, acc : 1.000, val_acc : 0.987
test_acc : 0.989
